In [ ]:
! git clone https://github.com/alumae/online_speaker_change_detector.git

In [ ]:
! pip install -r online_speaker_change_detector/requirements.txt

In [ ]:
import sys
sys.path.insert(1, "online_speaker_change_detector")

In [ ]:
import numpy as np

In [ ]:
from online_scd.utils import load_wav_file

In [ ]:
audio_full = load_wav_file("online_speaker_change_detector/test/sample_dataset/71_ID117_344945.wav", 16000)

In [ ]:
# extract around 2 minutes of audio, and remove the start which contains a jingle
audio = audio_full[int(5.88*16000) : 126*16000]

In [ ]:
audio

In [ ]:
import IPython.display as ipd

In [ ]:
ipd.Audio(audio, rate=16000)

In [ ]:
from online_scd.model import SCDModel
from online_scd.streaming import StreamingDecoder

model = SCDModel.load_from_checkpoint("online_speaker_change_detector/test/sample_model/checkpoints/epoch=102.ckpt")

In [ ]:
streaming_decoder = StreamingDecoder(model)
speaker_change_points = []
# stream in 1000 sample chunks, varying size chunks are accepted
for i in range(0, len(audio), 1000):
    for time in streaming_decoder.find_speaker_change_times(audio[i: i+1000]):
        print("Found speaker change point: ", time)
        speaker_change_points.append(time)


In [ ]:
# add short white noise clips to speaker change points

for p in speaker_change_points:
    samples = int(p * 16000)
    audio[samples - 100: samples+100] = np.random.random(200)

In [ ]:
# You can now hear the change points
ipd.Audio(audio, rate=16000)